<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/08.%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/78_GPU%E4%B8%8A%E3%81%A7%E3%81%AE%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題77のコードを改変し，GPU上で学習を実行せよ．

In [ ]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')
LOG_DIR = './logs'

# Check if GPU is available
# https://www.tensorflow.org/guide/gpu?hl=ja
tf.config.list_physical_devices('GPU')

Mounted at /content/drive


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
%load_ext tensorboard

In [ ]:
!python --version
!pip show google tensorflow

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: flatbuffers, opt-einsum, numpy, clang, keras, protobuf, wheel, wrapt, grpcio, typing-extensions, absl-py, tensorboard, termcolor, h5py, keras-preprocessing, tensorflow-estimator, six, astunparse, google-pasta, gast
Required-by: kapre


In [ ]:
def _parse_function(example_proto):
    # 特徴の記述
    feature_description = {
        'title': tf.io.FixedLenFeature([], tf.string),
        'category': tf.io.FixedLenFeature([], tf.string)}
  
  # 上記の記述を使って入力の tf.Example を処理
    features = tf.io.parse_single_example(example_proto, feature_description)
    X = tf.io.decode_raw(features['title'], tf.float32)
    y = tf.io.decode_raw(features['category'], tf.int32)
    return X, y

In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/'

def get_dataset(file_name):
    ds_raw = tf.data.TFRecordDataset(BASE_PATH+file_name+'.tfrecord')

    #shuffleはここを見て理解。データ件数取る方法がわからず、1000件に設定
    #https://qiita.com/exy81/items/d1388f6f02a11c8f1d7e
    return ds_raw.map(_parse_function).shuffle(1000).batch(32)

In [ ]:
train_ds = get_dataset('train')
valid_ds = get_dataset('valid')

In [ ]:
model = tf.keras.Sequential(
    [tf.keras.layers.Dense(
        4, activation='softmax', input_dim=300, use_bias=False, kernel_initializer='random_uniform') ])
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 1200      
Total params: 1,200
Trainable params: 1,200
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%tensorboard --logdir $LOG_DIR

In [ ]:
callbacks = []
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR))
#callbacks.append(tf.keras.callbacks.ModelCheckpoint('./tmp/{epoch:02d}-{val_loss:.2f}'))

In [ ]:
%%time
model.fit(train_ds, 
          epochs=30, 
          validation_data=valid_ds, 
          callbacks=callbacks)

# GPU 30 epochs
#CPU times: user 56.6 s, sys: 13.7 s, total: 1min 10s
#Wall time: 1min 4s

# CPU 30 epochs
#CPU times: user 32.1 s, sys: 6.74 s, total: 38.8 s
#Wall time: 35.6 s

Epoch 1/30
334/334 [==============================] - 7s 8ms/step - loss: 1.2569 - acc: 0.7012 - val_loss: 1.1444 - val_acc: 0.7769
Epoch 2/30
334/334 [==============================] - 2s 5ms/step - loss: 1.0722 - acc: 0.7788 - val_loss: 1.0114 - val_acc: 0.7859
Epoch 3/30
334/334 [==============================] - 2s 4ms/step - loss: 0.9680 - acc: 0.7785 - val_loss: 0.9292 - val_acc: 0.7859
Epoch 4/30
334/334 [==============================] - 2s 5ms/step - loss: 0.8996 - acc: 0.7781 - val_loss: 0.8721 - val_acc: 0.7852
Epoch 5/30
334/334 [==============================] - 2s 5ms/step - loss: 0.8502 - acc: 0.7785 - val_loss: 0.8294 - val_acc: 0.7859
Epoch 6/30
334/334 [==============================] - 2s 5ms/step - loss: 0.8120 - acc: 0.7780 - val_loss: 0.7955 - val_acc: 0.7867
Epoch 7/30
334/334 [==============================] - 2s 5ms/step - loss: 0.7811 - acc: 0.7785 - val_loss: 0.7676 - val_acc: 0.7859
Epoch 8/30
334/334 [==============================] - 2s 5ms/step - loss: 0.